## steps involved in nifty 50 stock portfolio without rebalance with heikein ashi OHLC data


- import necessary libraries
- Get the nifty 50 stock list
- Get nifty 50 stock price data from directory using glob library
- Write function to convert OHLC data into Heikein ashi data
- Calculate the technical Indicator using pandas_ta
- Write a crossover strategy for getting trading signal
- Write a portfolio stock performance to measure the performance of intraday strategy
- calling every above written function and print the every nifty 50 stock daily return without rebalance
- store the result into csv format or store the result into dictionary format.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import yfinance as yf
import datetime
import os
import glob
import pandas_ta as ta
import math
import time

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)

In [2]:
symbol_path = "D:\\stock_analysis_Spyder\\portfolio\\nifty_data\\ind_nifty50list.csv"
nifty_list_df = pd.read_csv(symbol_path)['Symbol']
nifty_list_df

0     ADANIPORTS
1     ASIANPAINT
2       AXISBANK
3     BAJAJ-AUTO
4     BAJFINANCE
5     BAJAJFINSV
6           BPCL
7     BHARTIARTL
8      BRITANNIA
9          CIPLA
10     COALINDIA
11      DIVISLAB
12       DRREDDY
13     EICHERMOT
14          GAIL
15        GRASIM
16       HCLTECH
17      HDFCBANK
18      HDFCLIFE
19    HEROMOTOCO
20      HINDALCO
21    HINDUNILVR
22          HDFC
23     ICICIBANK
24           ITC
25           IOC
26    INDUSINDBK
27          INFY
28      JSWSTEEL
29     KOTAKBANK
30            LT
31           M&M
32        MARUTI
33          NTPC
34     NESTLEIND
35          ONGC
36     POWERGRID
37      RELIANCE
38       SBILIFE
39      SHREECEM
40          SBIN
41     SUNPHARMA
42           TCS
43    TATAMOTORS
44     TATASTEEL
45         TECHM
46         TITAN
47           UPL
48    ULTRACEMCO
49         WIPRO
Name: Symbol, dtype: object

In [3]:
nifty_list = nifty_list_df.tolist()
nifty_list

['ADANIPORTS',
 'ASIANPAINT',
 'AXISBANK',
 'BAJAJ-AUTO',
 'BAJFINANCE',
 'BAJAJFINSV',
 'BPCL',
 'BHARTIARTL',
 'BRITANNIA',
 'CIPLA',
 'COALINDIA',
 'DIVISLAB',
 'DRREDDY',
 'EICHERMOT',
 'GAIL',
 'GRASIM',
 'HCLTECH',
 'HDFCBANK',
 'HDFCLIFE',
 'HEROMOTOCO',
 'HINDALCO',
 'HINDUNILVR',
 'HDFC',
 'ICICIBANK',
 'ITC',
 'IOC',
 'INDUSINDBK',
 'INFY',
 'JSWSTEEL',
 'KOTAKBANK',
 'LT',
 'M&M',
 'MARUTI',
 'NTPC',
 'NESTLEIND',
 'ONGC',
 'POWERGRID',
 'RELIANCE',
 'SBILIFE',
 'SHREECEM',
 'SBIN',
 'SUNPHARMA',
 'TCS',
 'TATAMOTORS',
 'TATASTEEL',
 'TECHM',
 'TITAN',
 'UPL',
 'ULTRACEMCO',
 'WIPRO']

In [4]:
portfolio_data_path = glob.glob('D:\\stock_analysis_JupyterNoteBook\\portfolio\\yfinance_data_porfolio\\yf_data\\2021-05-1_to_2021-05-30_5m\\*.csv')

In [5]:
portfolio_data_path

['D:\\stock_analysis_JupyterNoteBook\\portfolio\\yfinance_data_porfolio\\yf_data\\2021-05-1_to_2021-05-30_5m\\ADANIPORTS.NS.csv',
 'D:\\stock_analysis_JupyterNoteBook\\portfolio\\yfinance_data_porfolio\\yf_data\\2021-05-1_to_2021-05-30_5m\\ASIANPAINT.NS.csv',
 'D:\\stock_analysis_JupyterNoteBook\\portfolio\\yfinance_data_porfolio\\yf_data\\2021-05-1_to_2021-05-30_5m\\AXISBANK.NS.csv',
 'D:\\stock_analysis_JupyterNoteBook\\portfolio\\yfinance_data_porfolio\\yf_data\\2021-05-1_to_2021-05-30_5m\\BAJAJ-AUTO.NS.csv',
 'D:\\stock_analysis_JupyterNoteBook\\portfolio\\yfinance_data_porfolio\\yf_data\\2021-05-1_to_2021-05-30_5m\\BAJAJFINSV.NS.csv',
 'D:\\stock_analysis_JupyterNoteBook\\portfolio\\yfinance_data_porfolio\\yf_data\\2021-05-1_to_2021-05-30_5m\\BAJFINANCE.NS.csv',
 'D:\\stock_analysis_JupyterNoteBook\\portfolio\\yfinance_data_porfolio\\yf_data\\2021-05-1_to_2021-05-30_5m\\BHARTIARTL.NS.csv',
 'D:\\stock_analysis_JupyterNoteBook\\portfolio\\yfinance_data_porfolio\\yf_data\\2021-05-1_

In [6]:
def heikein_ashi(data):
    hadf = data.copy()
    #Close of heikein_ashi
    #hadf['cs_close'] = data['Close']
    hadf['Close'] = (data['Open'] + data['High'] + data['Low'] + data['Close'])/4

    #Open of heikein_ashi
    for i in range(len(hadf)):
        if i == 0:
            hadf['Open'].iat[i] = (data['Open'].iloc[i] + data['Close'].iloc[i])/2
        else:
            hadf['Open'].iat[i] = (hadf['Open'].iloc[i-1] + hadf['Close'].iloc[i-1])/2

    #Low of heikein_ashi
    hadf['Low'] = (hadf.loc[:,['Open','Close']].join(data['Low'])).min(axis=1)

    #High of heikein_ashi
    hadf['High'] = (hadf.loc[:,['Open','Close']].join(data['High'])).max(axis=1)
    hadf.insert(5,'cs_close',data['Close'])
    #print(f'Day of data: {np.unique(hadf.index.date)}') 
    return hadf

In [7]:
df = pd.read_csv(portfolio_data_path[35],index_col=0,parse_dates=True)

In [8]:
df.head()

,symbol,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,,
2021-05-03 09:15:00+05:30,ONGC,108.650002,110.699997,108.449997,109.949997,109.949997,0
2021-05-03 09:20:00+05:30,ONGC,110.000000,110.000000,106.750000,106.900002,106.900002,3256604
2021-05-03 09:25:00+05:30,ONGC,106.900002,107.349998,106.150002,107.099998,107.099998,1474260
2021-05-03 09:30:00+05:30,ONGC,106.800003,107.199997,106.050003,106.199997,106.199997,1329892
2021-05-03 09:35:00+05:30,ONGC,106.050003,106.750000,106.000000,106.650002,106.650002,833741


In [9]:
hadf = heikein_ashi(df)

In [10]:
hadf

,symbol,Open,High,Low,Close,cs_close,Adj Close,Volume
Datetime,,,,,,,,
2021-05-03 09:15:00+05:30,ONGC,109.299999,110.699997,108.449997,109.437498,109.949997,109.949997,0
2021-05-03 09:20:00+05:30,ONGC,109.368749,110.000000,106.750000,108.412500,106.900002,106.900002,3256604
2021-05-03 09:25:00+05:30,ONGC,108.890625,108.890625,106.150002,106.875000,107.099998,107.099998,1474260
2021-05-03 09:30:00+05:30,ONGC,107.882812,107.882812,106.050003,106.562500,106.199997,106.199997,1329892
2021-05-03 09:35:00+05:30,ONGC,107.222656,107.222656,106.000000,106.362501,106.650002,106.650002,833741
2021-05-03 09:40:00+05:30,ONGC,106.792579,107.550003,106.650002,107.112501,107.550003,107.550003,1207549
2021-05-03 09:45:00+05:30,ONGC,106.952540,107.800003,106.952540,107.487499,107.449997,107.449997,714085
2021-05-03 09:50:00+05:30,ONGC,107.220020,107.900002,107.220020,107.487501,107.300003,107.300003,412143
2021-05-03 09:55:00+05:30,ONGC,107.353760,107.353760,106.900002,107.175001,107.199997,107.199997,447223


In [11]:
date_time = np.unique(hadf.index.date)
date_time

array([datetime.date(2021, 5, 3), datetime.date(2021, 5, 4),
       datetime.date(2021, 5, 5), datetime.date(2021, 5, 6),
       datetime.date(2021, 5, 7), datetime.date(2021, 5, 10),
       datetime.date(2021, 5, 11), datetime.date(2021, 5, 12),
       datetime.date(2021, 5, 14), datetime.date(2021, 5, 17),
       datetime.date(2021, 5, 18), datetime.date(2021, 5, 19),
       datetime.date(2021, 5, 20), datetime.date(2021, 5, 21),
       datetime.date(2021, 5, 24), datetime.date(2021, 5, 25),
       datetime.date(2021, 5, 26), datetime.date(2021, 5, 27),
       datetime.date(2021, 5, 28)], dtype=object)

In [12]:
def macd(data,fast=12,slow=26,signal=9):
    data = ta.macd(data['Close'],fast=fast,slow=slow,signal=signal)
    data.rename(columns={data.columns[0]:'macd',data.columns[1]:'histogram',data.columns[2]:'macd_signal'},inplace=True)
    data = data[['macd','macd_signal']]
    return data

def supertrend(data,length=10,multiplier=1):
    data = ta.supertrend(high=data['High'],low=data['Low'],close=data['Close'],length=length,multiplier=multiplier)
    data.rename(columns={data.columns[0]:'supertrend'},inplace=True)
    data = data.loc[:,[data.columns[0]]]
    return data

def combine_macd_st(price_data,macd_data,supertrend_data):
    data = pd.concat([price_data,macd_data,supertrend_data],axis=1)
    return data

#combine ST and macd
def strategy_technical_indicator(data,fast=12,slow=26,signal=9,length=100,multiplier=1):
    hadf_macd = macd(data=data,fast=fast,slow=slow,signal=signal)
    hadf_supertrend = supertrend(data=data,length=length,multiplier=multiplier)
    final_hadf = combine_macd_st(data[['symbol','Close']],hadf_macd,hadf_supertrend)
    final_hadf = final_hadf.round(2)
    return final_hadf

In [13]:
#~~~ python
def supertrend_crossover(data,capital):
        
    account_balance = round(capital,2)
    data['quantity'] = 0
    data['signal'] = " "
    data['position'] = 0
    data['account_balance'] = np.nan
    data['capital_invested'] = 0
    data['trade_price'] = 0
    data['trade_return'] = 0
   
    
    data['PL'] = 0
    
    for i in range(1,len(data.index)):
        if data['position'].iloc[i] == 0:
            data['account_balance'].iloc[i] = account_balance
        # carry position
        if data['position'].iloc[i-1] != 0:
            data['position'].iloc[i] = data['position'].iloc[i-1]
            data['trade_price'].iloc[i] = data['trade_price'].iloc[i-1]
            data['quantity'].iloc[i] = data['quantity'].iloc[i-1]
            data['account_balance'].iloc[i] = data['account_balance'].iloc[i-1]
            data['capital_invested'].iloc[i] = data['capital_invested'].iloc[i-1]
            #print(i,str(data.index[i]),'carry position')
        
        #entring a new position
        if data['position'].iloc[i] == 0 and account_balance >0:
            trade_price = data['Close'].iloc[i]
            quantity = math.floor(data['account_balance'].iloc[i]/trade_price)
            if quantity != 0:
                #entry long position
                if data['macd'].iloc[i] > data['macd_signal'].iloc[i] and data['Close'].iloc[i] > data['supertrend'].iloc[i]:
                    data['position'].iloc[i] = 1
                    data['trade_price'].iloc[i] = trade_price
                    data['signal'].iloc[i] = "Entry long"
               
                    #trading quantity and value
                    data['account_balance'].iloc[i] = account_balance
                    data['quantity'].iloc[i] = quantity
                    data['capital_invested'].iloc[i] = quantity * trade_price
                    #print(i,str(data.index[i]),'Entry Long Price : ',round(data['Close'].iloc[i]),2)
           
                #entry short position
                if data['macd'].iloc[i] < data['macd_signal'].iloc[i] and data['Close'].iloc[i] < data['supertrend'].iloc[i]:
                    data['position'].iloc[i] = -1
                    data['trade_price'].iloc[i] = data['Close'].iloc[i]
                    data['signal'].iloc[i] = "Entry short"
            
                    #trading quantity and value
                    data['account_balance'].iloc[i] = account_balance
                    data['quantity'].iloc[i] = quantity 
                    data['capital_invested'].iloc[i] = quantity * trade_price
                    #print(i,str(data.index[i]),'Entry Short Price :',round(data['Close'].iloc[i]),2)
                
        #exiting long position 
        if data['position'].iloc[i] == 1:
            if data['Close'].iloc[i] < data['supertrend'].iloc[i]:
                data['position'].iloc[i] = 0
                trade_return = (data['Close'].iloc[i] - data['trade_price'].iloc[i])/data['trade_price'].iloc[i]
                data['trade_return'].iloc[i] = trade_return
                #data['trade_price'].iloc[i] = data['Close'].iloc[i]
                data['signal'].iloc[i] = "Exit long position"
                
                #trading quantity
                PL = trade_return * data['capital_invested'].iloc[i-1]
                data['PL'].iloc[i] = PL
                data['capital_invested'].iloc[i] = PL + data['capital_invested'].iloc[i]
                trade_balance = account_balance + PL
                data['account_balance'].iloc[i] = trade_balance
                account_balance = trade_balance
                data['quantity'].iloc[i]=0 
                #print(i,str(data.index[i]),'Exit long Price :',round(data['Close'].iloc[i],2),'P&L = %.2f'%account_balance)
        
        #exiting short position        
        if data['position'].iloc[i] == -1:
            if data['Close'].iloc[i] > data['supertrend'].iloc[i]:
                data['position'].iloc[i] = 0
                trade_return = (data['trade_price'].iloc[i] - data['Close'].iloc[i])/data['trade_price'].iloc[i]
                data['trade_return'].iloc[i] = trade_return 
                #data['trade_price'].iloc[i] = data['Close'].iloc[i]
                data['signal'].iloc[i] = "Exit short Position"
            
                #trading quantity
                PL = trade_return * data['capital_invested'].iloc[i-1]
                data['PL'].iloc[i] = PL
                data['capital_invested'].iloc[i] = PL + data['capital_invested'].iloc[i]
                trade_balance = account_balance + PL
                data['account_balance'].iloc[i] = trade_balance
                account_balance = trade_balance
                data['quantity'].iloc[i]=0               
                #print(i,str(data.index[i]),'Exit short Price :',round(data['Close'].iloc[i],2),'P&L = %.2f'%account_balance)
                
        # Closing Trade at 15:00
        if data.index[i].time() >= datetime.time(15,0):
            #closing LONG trade
            if data['position'].iloc[i]==1:
                data['position'].iloc[i] = 0
                trade_return = (data['Close'].iloc[i] - data['trade_price'].iloc[i])/data['trade_price'].iloc[i]
                data['trade_return'].iloc[i] = trade_return
                #data['trade_price'].iloc[i] = data['Close'].iloc[i]
                data['signal'].iloc[i] = "Trade close with Long position"
                
                #trading quantity
                PL = trade_return * data['capital_invested'].iloc[i-1]
                data['PL'].iloc[i] = PL
                data['capital_invested'].iloc[i] = PL + data['capital_invested'].iloc[i]
                trade_balance = account_balance + PL
                data['account_balance'].iloc[i] = trade_balance
                account_balance = trade_balance
                data['quantity'].iloc[i]=0 
                
                #print(i,str(data.index[i]),'TRADE Close WITH LONG EXIT :',round(data['Close'].iloc[i],2),'P&L = %.2f'%account_balance)
                break
            
            # closing SHORT trade
            if data['position'].iloc[i] == -1:
                data['position'].iloc[i] = 0
                trade_return = (data['trade_price'].iloc[i] - data['Close'].iloc[i])/data['trade_price'].iloc[i]
                data['trade_return'].iloc[i] = trade_return 
                #data['trade_price'].iloc[i] = data['Close'].iloc[i]
                data['signal'].iloc[i] = "Trade close with short Position"
            
                #trading quantity
                PL = trade_return * data['capital_invested'].iloc[i-1]
                data['PL'].iloc[i] = PL
                data['capital_invested'].iloc[i] = PL + data['capital_invested'].iloc[i]
                trade_balance = account_balance + PL
                data['account_balance'].iloc[i] = trade_balance
                account_balance = trade_balance
                data['quantity'].iloc[i]=0  
                #print(i,str(data.index[i]),'TRADE Close WITH SHORT EXIT :',round(data['Close'].iloc[i],2),'P&L = %.2f'%account_balance)
                break
            
    #data['cummulative_return'] = data['trade_return'].cumsum()
    #print('Total_return = {}'.format(data['cummulative_return'][-1]))
    #print('Total P&L= %.2f'%account_balance)
    return data
#~~~

In [14]:
def portfolio_stock_performance(data):
    profit = data[data['trade_return']>0]['trade_return'].tolist()
    loss = data[data['trade_return']<0]['trade_return'].tolist()
    Total_profits = round(np.sum(profit),3)
    Total_losses = round(np.abs(np.sum(loss)),3)
    total_return = data['trade_return'].sum()
    capital_invested = data['account_balance'].iloc[1]
    pl = data['PL'].sum()
    
    perform_m = {}
    perform_m['date'] = data.index.date[0]
    perform_m['symbol'] = data['symbol'].iloc[0]
    perform_m['Total_win'] = data[data['trade_return']>0]['trade_return'].count() ##avoid creating new column
    perform_m['Total_loss'] = data[data['trade_return']<0]['trade_return'].count()
    perform_m['No_of_trades'] = perform_m['Total_win']+perform_m['Total_loss']
    perform_m['max_quantity'] = max(np.unique(data['quantity']))
    perform_m['Total Return(%)']=round(total_return,4)
    capital_return = data.loc[datetime.time(15,0),'account_balance'].get_values()[0]
    perform_m['capital_invested'] = capital_invested
    perform_m['capital_return'] = capital_return
    perform_m['PL'] = pl
    portfolio_metrics = pd.Series(perform_m) 
    return portfolio_metrics.to_frame().T

In [15]:
stored_stock_list = ['Date']

for count,symbol in enumerate(portfolio_data_path[:-1]):
    stock = symbol.split('\\')[-1].split('.')[0]
    #print(count,stock)
    stored_stock_list.append(stock)

stored_stock_list.append('Total_Return')

stored_stock_list

['Date',
 'ADANIPORTS',
 'ASIANPAINT',
 'AXISBANK',
 'BAJAJ-AUTO',
 'BAJAJFINSV',
 'BAJFINANCE',
 'BHARTIARTL',
 'BPCL',
 'BRITANNIA',
 'CIPLA',
 'COALINDIA',
 'DIVISLAB',
 'DRREDDY',
 'EICHERMOT',
 'GAIL',
 'GRASIM',
 'HCLTECH',
 'HDFC',
 'HDFCBANK',
 'HDFCLIFE',
 'HEROMOTOCO',
 'HINDALCO',
 'HINDUNILVR',
 'ICICIBANK',
 'INDUSINDBK',
 'INFY',
 'IOC',
 'ITC',
 'JSWSTEEL',
 'KOTAKBANK',
 'LT',
 'M&M',
 'MARUTI',
 'NESTLEIND',
 'NTPC',
 'ONGC',
 'POWERGRID',
 'RELIANCE',
 'SBILIFE',
 'SBIN',
 'SHREECEM',
 'SUNPHARMA',
 'TATAMOTORS',
 'TATASTEEL',
 'TCS',
 'TECHM',
 'TITAN',
 'ULTRACEMCO',
 'UPL',
 'WIPRO',
 'Total_Return']

~~~ python
res_columns = ['date', 'symbol', 'Total_win', 'Total_loss', 'Total_number_of_trades',
       'max_quantity', 'Total Return(%)', 'capital_invested',
       'capital_return']
res_dataframe = pd.DataFrame(columns=res_columns)
res_dataframe

port_df = pd.DataFrame(columns=['Date','ONGC','quant','Return'])

datewise_strategy_dict = {}

weights = [10000,10000,10398.09,9797.24,10258.50,10488.93,10412.33,13593.60,19279.84,18547.19,20351.41,24014.29,27583.31]

for count,(dt,allocation) in enumerate(zip(date_time,weights)):
    port_df.loc[count] = 0.0
    port_df['Date'].iloc[count] = dt
    print('\n{} = ₹ {}'.format(str(dt),allocation))
    #port_df['quant'].iloc[count] = 0
    hadf = heikein_ashi(df)
    calc_all_day_df =strategy_technical_indicator(hadf)
    one_day_calc_df = calc_all_day_df[calc_all_day_df.index.date == dt]
    strategy_df = supertrend_crossover(one_day_calc_df,capital=allocation)
    
    datewise_strategy_dict[str(dt)] = strategy_df
    
    strategy_result = portfolio_stock_performance(strategy_df)  
    res_dataframe = res_dataframe.append(strategy_result,ignore_index=True)
    #print(strategy_result)
    stock_PL = strategy_result['capital_return'].iloc[0]
    #allocation = stock_PL
    max_quantity = strategy_result['max_quantity'].iloc[0]
    day_close_price = strategy_df['Close'][-1]
    print('\t{:<4} {:<10} close_price = {:<6} | quant = {:<3} | PL = {:<10}'.format(count,str(dt),day_close_price,max_quantity,stock_PL))
    port_df['quant'].iloc[count] = max_quantity
    port_df['ONGC'].iloc[count] = stock_PL
    port_df['Return'].iloc[count] = strategy_result['Total Return(%)'].iloc[0]
~~~

In [16]:
##creating empty dataframe
res_columns = ['date', 'symbol', 'Total_win', 'Total_loss', 'No_of_trades',
       'max_quantity', 'Total Return(%)', 'capital_invested',
       'capital_return','PL']
allday_port_result = pd.DataFrame(columns=res_columns)    ## dataframe
## create empty portfolio dataframe
port_df = pd.DataFrame(columns=stored_stock_list)         ## dataframe
portfolio_strategy_dict = {}                              ## dict
oneday_port_result_dict = {}                                     ## dict
print("Portfolio start in: {}".format(datetime.datetime.now().replace(microsecond=0)))
portfolio_start = time.perf_counter()

account_balance = 500000  #total capital invested
allocation = 10000   #allocation 10000 per stock in nifty50 company
for dt_count,dt in enumerate(date_time):
    oneday_port_df = pd.DataFrame(columns=res_columns)     ## dataframe
    port_df.loc[dt_count] = 0.0
    port_df['Date'].iloc[dt_count] = dt
    print('\n')
    per_date_start = time.perf_counter()
    print("Start Day {} : \033[1m{}\033[0m => start in: {}".format(dt_count,dt,datetime.datetime.now().replace(microsecond=0)))
    print("\t\033[1m{}\033[0m Equal weighted Per stock allocation: \033[1m\033[34m₹ {:.2f}\033[0m".format(dt,allocation))
    
    portfolio_strategy_dict[str(dt)] = {}
    for count,stock in enumerate(portfolio_data_path[:-1]):
        stock_start = time.perf_counter()
        symbol = stock.split('\\')[-1].split('.')[0]
        raw_data = pd.read_csv(stock,index_col=0,parse_dates=True)
        hadf = heikein_ashi(raw_data)
        calc_all_day_df = strategy_technical_indicator(hadf)
        one_day_calc_df = calc_all_day_df[calc_all_day_df.index.date == dt]
        strategy_df = supertrend_crossover(one_day_calc_df,capital=allocation)
        
        day_close_price = strategy_df['Close'][-1]
        strategy_result = portfolio_stock_performance(strategy_df)
        
        allday_port_result = allday_port_result.append(strategy_result,ignore_index=True)
        oneday_port_df = oneday_port_df.append(strategy_result,ignore_index=True)
        
        portfolio_strategy_dict[str(dt)][symbol] = strategy_df
        #portfolio_strategy_dict.update([str(dt)][symbol]) = strategy_df
        percent_return = strategy_result['Total Return(%)'].iloc[0]
        stock_PL = strategy_result['capital_return'].iloc[0]
        max_quantity = strategy_result['max_quantity'].iloc[0]
        pl = round(strategy_result['PL'].iloc[0],2)
        port_df[symbol].iloc[dt_count] = stock_PL
        stock_end = time.perf_counter()
        stock_time_taken = stock_end - stock_start
        print('\t{:<4} {:<10} : LTP = {:<10} | Quant = {:<2} | %_ret= {:<5}| Ret= ₹ {:<10.2f} | PL = ₹ {:<6} | TT= {}'.format(count,symbol,day_close_price,max_quantity,percent_return,stock_PL,pl,time.strftime("%H:%M:%S",time.gmtime(stock_time_taken))))
    total_day_return = port_df.sum(axis=1).iloc[dt_count]
    
    port_df['Total_Return'].iloc[dt_count] = total_day_return
    
    oneday_port_result_dict[str(dt)] =  oneday_port_df
    
    account_balance = total_day_return
    allocation = total_day_return/50  ############change
    
    print("End Day {} : \033[1m{}\033[0m => end in: {}".format(dt_count,dt,datetime.datetime.now().replace(microsecond=0)))
    per_date_stop = time.perf_counter()
    per_date_time_taken = per_date_stop - per_date_start  
    print("\033[31m-\033[0m"*60)
    print("\033[1m{} : time taken: {} | \033[31mTotal Day Return: ₹ {:.2f}\033[0m".format(dt,time.strftime("%H:%M:%S",time.gmtime(per_date_time_taken)),total_day_return))
    till_now_portfolio = time.perf_counter()
    portfolio_time_after_stock = till_now_portfolio - portfolio_start
    print("\033[31m-\033[0m"*60)
    #print("Day {} Portfolio in: {}".format(dt_count,datetime.datetime.now().replace(microsecond=0)))
    print("Portfolio time taken after = day {} : {}".format(dt,time.strftime("%H:%M:%S",time.gmtime(portfolio_time_after_stock))))
    print("\033[34m=\033[0m"*125)
portfolio_end = time.perf_counter()
print("\nPortfolio end in: {}".format(datetime.datetime.now().replace(microsecond=0)))
portfolio_time_taken = portfolio_end - portfolio_start
print("Whole portfolio Time_taken: {} | \t\033[1m\033[34mTotal Portfolio Return: ₹ {:.2f}\033[0m\033[0m".format(time.strftime("%H:%M:%S",time.gmtime(portfolio_time_taken)),total_day_return))

Portfolio start in: 2021-06-08 04:37:52


Start Day 0 : 2021-05-03 => start in: 2021-06-08 04:37:52
	2021-05-03 Equal weighted Per stock allocation: ₹ 10000.00
	0    ADANIPORTS : LTP = 762.71     | Quant = 0  | Ret= ₹ 10000.00   | PL = ₹0        | TT= 00:00:06
	1    ASIANPAINT : LTP = 2586.1     | Quant = 0  | Ret= ₹ 10000.00   | PL = ₹0        | TT= 00:00:06
	2    AXISBANK   : LTP = 703.8      | Quant = 0  | Ret= ₹ 10000.00   | PL = ₹0        | TT= 00:00:06
	3    BAJAJ-AUTO : LTP = 3851.39    | Quant = 0  | Ret= ₹ 10000.00   | PL = ₹0        | TT= 00:00:07
	4    BAJAJFINSV : LTP = 11037.19   | Quant = 0  | Ret= ₹ 10000.00   | PL = ₹0        | TT= 00:00:06
	5    BAJFINANCE : LTP = 5549.49    | Quant = 0  | Ret= ₹ 10000.00   | PL = ₹0        | TT= 00:00:06
	6    BHARTIARTL : LTP = 561.39     | Quant = 0  | Ret= ₹ 10000.00   | PL = ₹0        | TT= 00:00:06
	7    BPCL       : LTP = 416.16     | Quant = 0  | Ret= ₹ 10000.00   | PL = ₹0        | TT= 00:00:06
	8    BRITANNIA  : LTP = 3440.14

	2    AXISBANK   : LTP = 700.57     | Quant = 14 | Ret= ₹ 10158.48   | PL = ₹158.48   | TT= 00:00:13
	3    BAJAJ-AUTO : LTP = 3834.03    | Quant = 2  | Ret= ₹ 9967.48    | PL = ₹-32.52   | TT= 00:00:28
	4    BAJAJFINSV : LTP = 10994.1    | Quant = 0  | Ret= ₹ 10000.00   | PL = ₹0        | TT= 00:00:07
	5    BAJFINANCE : LTP = 5605.32    | Quant = 1  | Ret= ₹ 10033.36   | PL = ₹33.36    | TT= 00:00:22
	6    BHARTIARTL : LTP = 551.99     | Quant = 18 | Ret= ₹ 10071.77   | PL = ₹71.77    | TT= 00:00:24
	7    BPCL       : LTP = 422.19     | Quant = 23 | Ret= ₹ 10003.91   | PL = ₹3.91     | TT= 00:00:27
	8    BRITANNIA  : LTP = 3435.28    | Quant = 2  | Ret= ₹ 9999.92    | PL = ₹-0.08    | TT= 00:00:25
	9    CIPLA      : LTP = 881.5      | Quant = 11 | Ret= ₹ 9883.62    | PL = ₹-116.38  | TT= 00:00:24
	10   COALINDIA  : LTP = 132.75     | Quant = 75 | Ret= ₹ 10019.40   | PL = ₹19.4     | TT= 00:00:24
	11   DIVISLAB   : LTP = 3991.18    | Quant = 2  | Ret= ₹ 10136.68   | PL = ₹136.68   | TT=

	6    BHARTIARTL : LTP = 559.85     | Quant = 17 | Ret= ₹ 10034.56   | PL = ₹0.85     | TT= 00:00:31
	7    BPCL       : LTP = 425.01     | Quant = 23 | Ret= ₹ 10009.10   | PL = ₹-24.61   | TT= 00:00:32
	8    BRITANNIA  : LTP = 3451.19    | Quant = 2  | Ret= ₹ 10026.87   | PL = ₹-6.84    | TT= 00:00:33
	9    CIPLA      : LTP = 889.79     | Quant = 11 | Ret= ₹ 10146.46   | PL = ₹112.75   | TT= 00:00:31
	10   COALINDIA  : LTP = 133.41     | Quant = 75 | Ret= ₹ 9889.09    | PL = ₹-144.62  | TT= 00:00:35
	11   DIVISLAB   : LTP = 4040.19    | Quant = 2  | Ret= ₹ 10039.61   | PL = ₹5.9      | TT= 00:00:34
	12   DRREDDY    : LTP = 5163.36    | Quant = 1  | Ret= ₹ 10059.56   | PL = ₹25.85    | TT= 00:00:32
	13   EICHERMOT  : LTP = 2374.55    | Quant = 4  | Ret= ₹ 10038.99   | PL = ₹5.28     | TT= 00:00:36
	14   GAIL       : LTP = 145.33     | Quant = 69 | Ret= ₹ 9994.04    | PL = ₹-39.67   | TT= 00:00:27
	15   GRASIM     : LTP = 1420.44    | Quant = 7  | Ret= ₹ 10205.94   | PL = ₹172.23   | TT=

	10   COALINDIA  : LTP = 134.53     | Quant = 74 | Ret= ₹ 10060.75   | PL = ₹25.16    | TT= 00:00:32
	11   DIVISLAB   : LTP = 4063.8     | Quant = 2  | Ret= ₹ 9995.73    | PL = ₹-39.86   | TT= 00:00:34
	12   DRREDDY    : LTP = 5172.17    | Quant = 1  | Ret= ₹ 10064.36   | PL = ₹28.77    | TT= 00:00:33
	13   EICHERMOT  : LTP = 2437.01    | Quant = 4  | Ret= ₹ 10154.23   | PL = ₹118.64   | TT= 00:00:26
	14   GAIL       : LTP = 151.25     | Quant = 68 | Ret= ₹ 9979.10    | PL = ₹-56.49   | TT= 00:00:34
	15   GRASIM     : LTP = 1447.35    | Quant = 7  | Ret= ₹ 9981.91    | PL = ₹-53.68   | TT= 00:00:34
	16   HCLTECH    : LTP = 915.54     | Quant = 11 | Ret= ₹ 10102.94   | PL = ₹67.35    | TT= 00:00:33
	17   HDFC       : LTP = 2431.62    | Quant = 4  | Ret= ₹ 10075.75   | PL = ₹40.16    | TT= 00:00:28
	18   HDFCBANK   : LTP = 1402.06    | Quant = 7  | Ret= ₹ 9956.00    | PL = ₹-79.59   | TT= 00:00:35
	19   HDFCLIFE   : LTP = 672.1      | Quant = 14 | Ret= ₹ 9991.91    | PL = ₹-43.68   | TT=

	14   GAIL       : LTP = 149.95     | Quant = 66 | Ret= ₹ 9939.99    | PL = ₹-114.55  | TT= 00:00:34
	15   GRASIM     : LTP = 1448.82    | Quant = 6  | Ret= ₹ 10035.70   | PL = ₹-18.84   | TT= 00:00:36
	16   HCLTECH    : LTP = 915.86     | Quant = 10 | Ret= ₹ 10002.44   | PL = ₹-52.1    | TT= 00:00:36
	17   HDFC       : LTP = 2492.04    | Quant = 4  | Ret= ₹ 10133.06   | PL = ₹78.52    | TT= 00:00:34
	18   HDFCBANK   : LTP = 1413.04    | Quant = 7  | Ret= ₹ 10010.02   | PL = ₹-44.52   | TT= 00:00:36
	19   HDFCLIFE   : LTP = 673.04     | Quant = 14 | Ret= ₹ 9961.44    | PL = ₹-93.1    | TT= 00:00:36
	20   HEROMOTOCO : LTP = 2854.6     | Quant = 3  | Ret= ₹ 10274.92   | PL = ₹220.38   | TT= 00:00:34
	21   HINDALCO   : LTP = 401.15     | Quant = 25 | Ret= ₹ 10124.54   | PL = ₹70.0     | TT= 00:00:35
	22   HINDUNILVR : LTP = 2411.74    | Quant = 4  | Ret= ₹ 10026.98   | PL = ₹-27.56   | TT= 00:00:36
	23   ICICIBANK  : LTP = 607.09     | Quant = 16 | Ret= ₹ 10086.06   | PL = ₹31.52    | TT=

...

	46   TITAN      : LTP = 1535.44    | Quant = 6  | Ret= ₹ 10368.12   | PL = ₹61.8     | TT= 00:00:42
	47   ULTRACEMCO : LTP = 6575.65    | Quant = 1  | Ret= ₹ 10242.85   | PL = ₹-63.47   | TT= 00:00:46
	48   UPL        : LTP = 769.04     | Quant = 13 | Ret= ₹ 10397.84   | PL = ₹91.52    | TT= 00:00:42
	49   WIPRO      : LTP = 508.45     | Quant = 20 | Ret= ₹ 10331.57   | PL = ₹25.25    | TT= 00:00:43
End Day 12 : 2021-05-20 => end in: 2021-06-08 10:16:36
------------------------------------------------------------
2021-05-20 : time taken: 00:33:21 | Total Day Return: ₹ 516870.65
------------------------------------------------------------
Portfolio time taken after = day 2021-05-20 : 05:38:44


Start Day 13 : 2021-05-21 => start in: 2021-06-08 10:16:37
	2021-05-21 Equal weighted Per stock allocation: ₹ 10337.41
	0    ADANIPORTS : LTP = 769.2      | Quant = 13 | Ret= ₹ 10268.64   | PL = ₹-68.77   | TT= 00:00:45
	1    ASIANPAINT : LTP = 2831.92    | Quant = 3  | Ret= ₹ 10345.72   | PL = 

	0    ADANIPORTS : LTP = 766.15     | Quant = 13 | Ret= ₹ 10263.66   | PL = ₹-97.76   | TT= 00:00:43
	1    ASIANPAINT : LTP = 2819.31    | Quant = 3  | Ret= ₹ 10328.03   | PL = ₹-33.39   | TT= 00:00:45
	2    AXISBANK   : LTP = 738.5      | Quant = 14 | Ret= ₹ 10421.34   | PL = ₹59.92    | TT= 00:00:34
	3    BAJAJ-AUTO : LTP = 4146.56    | Quant = 2  | Ret= ₹ 10407.32   | PL = ₹45.9     | TT= 00:00:45
	4    BAJAJFINSV : LTP = 11150.61   | Quant = 0  | Ret= ₹ 10361.42   | PL = ₹0        | TT= 00:00:08
	5    BAJFINANCE : LTP = 5594.81    | Quant = 1  | Ret= ₹ 10344.59   | PL = ₹-16.83   | TT= 00:00:44
	6    BHARTIARTL : LTP = 526.73     | Quant = 19 | Ret= ₹ 10361.23   | PL = ₹-0.19    | TT= 00:00:44
	7    BPCL       : LTP = 473.29     | Quant = 22 | Ret= ₹ 10361.56   | PL = ₹0.14     | TT= 00:00:43
	8    BRITANNIA  : LTP = 3383.09    | Quant = 3  | Ret= ₹ 10464.05   | PL = ₹102.63   | TT= 00:00:42
	9    CIPLA      : LTP = 931.85     | Quant = 11 | Ret= ₹ 10389.91   | PL = ₹28.49    | TT=

	4    BAJAJFINSV : LTP = 11392.59   | Quant = 0  | Ret= ₹ 10379.30   | PL = ₹0        | TT= 00:00:08
	5    BAJFINANCE : LTP = 5636.53    | Quant = 1  | Ret= ₹ 10374.63   | PL = ₹-4.67    | TT= 00:00:46
	6    BHARTIARTL : LTP = 526.93     | Quant = 19 | Ret= ₹ 10374.55   | PL = ₹-4.75    | TT= 00:00:44
	7    BPCL       : LTP = 473.61     | Quant = 22 | Ret= ₹ 10199.25   | PL = ₹-180.05  | TT= 00:00:42
	8    BRITANNIA  : LTP = 3440.28    | Quant = 3  | Ret= ₹ 10424.00   | PL = ₹44.7     | TT= 00:00:45
	9    CIPLA      : LTP = 934.69     | Quant = 11 | Ret= ₹ 10373.14   | PL = ₹-6.16    | TT= 00:00:45
	10   COALINDIA  : LTP = 147.46     | Quant = 69 | Ret= ₹ 10440.02   | PL = ₹60.72    | TT= 00:00:45
	11   DIVISLAB   : LTP = 4101.74    | Quant = 2  | Ret= ₹ 10347.44   | PL = ₹-31.86   | TT= 00:00:46
	12   DRREDDY    : LTP = 5317.95    | Quant = 1  | Ret= ₹ 10351.68   | PL = ₹-27.62   | TT= 00:00:47
	13   EICHERMOT  : LTP = 2628.84    | Quant = 4  | Ret= ₹ 10315.22   | PL = ₹-64.08   | TT=

	8    BRITANNIA  : LTP = 3415.74    | Quant = 3  | Ret= ₹ 10391.00   | PL = ₹-13.65   | TT= 00:00:54
	9    CIPLA      : LTP = 945.95     | Quant = 11 | Ret= ₹ 10435.34   | PL = ₹30.69    | TT= 00:00:59
	10   COALINDIA  : LTP = 145.85     | Quant = 70 | Ret= ₹ 10492.85   | PL = ₹88.2     | TT= 00:00:57


KeyboardInterrupt: 

In [17]:
portfolio_strategy_dict.keys()

dict_keys(['2021-05-03', '2021-05-04', '2021-05-05', '2021-05-06', '2021-05-07', '2021-05-10', '2021-05-11', '2021-05-12', '2021-05-14', '2021-05-17', '2021-05-18', '2021-05-19', '2021-05-20', '2021-05-21', '2021-05-24', '2021-05-25', '2021-05-26'])

In [ ]:
portfolio_strategy_dict['2021-05-03'].keys()

In [18]:
portfolio_strategy_dict['2021-05-04']['SBIN']

,symbol,Close,macd,macd_signal,supertrend,quantity,signal,position,account_balance,trade_price,trade_return,capital_invested,PL
Datetime,,,,,,,,,,,,,
2021-05-04 09:15:00+05:30,SBIN,355.44,1.35,1.04,NaN,0,,0,NaN,0.00,0.000000,0.00,0.00
2021-05-04 09:20:00+05:30,SBIN,355.56,1.61,1.16,NaN,0,,0,10000.00,0.00,0.000000,0.00,0.00
2021-05-04 09:25:00+05:30,SBIN,355.04,1.75,1.28,NaN,0,,0,10000.00,0.00,0.000000,0.00,0.00
2021-05-04 09:30:00+05:30,SBIN,355.19,1.86,1.39,NaN,0,,0,10000.00,0.00,0.000000,0.00,0.00
2021-05-04 09:35:00+05:30,SBIN,356.03,1.98,1.51,NaN,0,,0,10000.00,0.00,0.000000,0.00,0.00
2021-05-04 09:40:00+05:30,SBIN,355.56,2.02,1.61,NaN,0,,0,10000.00,0.00,0.000000,0.00,0.00
2021-05-04 09:45:00+05:30,SBIN,354.59,1.95,1.68,NaN,0,,0,10000.00,0.00,0.000000,0.00,0.00
2021-05-04 09:50:00+05:30,SBIN,355.21,1.93,1.73,NaN,0,,0,10000.00,0.00,0.000000,0.00,0.00
2021-05-04 09:55:00+05:30,SBIN,356.76,2.01,1.79,NaN,0,,0,10000.00,0.00,0.000000,0.00,0.00


In [38]:
oneday_port_result_dict.keys()

dict_keys(['2021-05-03', '2021-05-04', '2021-05-05', '2021-05-06', '2021-05-07', '2021-05-10', '2021-05-11', '2021-05-12', '2021-05-14', '2021-05-17', '2021-05-18', '2021-05-19', '2021-05-20', '2021-05-21', '2021-05-24', '2021-05-25'])

In [39]:
oneday_port_result_dict

{'2021-05-03':           date      symbol Total_win Total_loss No_of_trades max_quantity  \
 0   2021-05-03  ADANIPORTS         0          0            0            0   
 1   2021-05-03  ASIANPAINT         0          0            0            0   
 2   2021-05-03    AXISBANK         0          0            0            0   
 3   2021-05-03  BAJAJ-AUTO         0          0            0            0   
 4   2021-05-03  BAJAJFINSV         0          0            0            0   
 5   2021-05-03  BAJFINANCE         0          0            0            0   
 6   2021-05-03  BHARTIARTL         0          0            0            0   
 7   2021-05-03        BPCL         0          0            0            0   
 8   2021-05-03   BRITANNIA         0          0            0            0   
 9   2021-05-03       CIPLA         0          0            0            0   
 10  2021-05-03   COALINDIA         0          0            0            0   
 11  2021-05-03    DIVISLAB         0          0  

In [36]:
oneday_port_df

,date,symbol,Total_win,Total_loss,No_of_trades,max_quantity,Total Return(%),capital_invested,capital_return,PL
0,2021-05-26,ADANIPORTS,2,2,4,13,0.0108,10404.6,10418.6,13.91
1,2021-05-26,ASIANPAINT,2,0,2,3,0.0039,10404.6,10436.9,32.22
2,2021-05-26,AXISBANK,2,1,3,14,0.0661,10404.6,10487.9,83.3
3,2021-05-26,BAJAJ-AUTO,3,0,3,2,0.0073,10404.6,10446.2,41.56
4,2021-05-26,BAJAJFINSV,0,0,0,0,0,10404.6,10404.6,0
5,2021-05-26,BAJFINANCE,3,1,4,1,0.0198,10404.6,10516.6,111.98
6,2021-05-26,BHARTIARTL,2,2,4,19,0.0774,10404.6,10433,28.31
7,2021-05-26,BPCL,2,6,8,21,-0.1335,10404.6,10221.1,-183.54
8,2021-05-26,BRITANNIA,1,2,3,3,0.0064,10404.6,10391,-13.65
9,2021-05-26,CIPLA,2,2,4,11,0.0032,10404.6,10435.3,30.69


In [ ]:
print(datetime.datetime.now().replace(microsecond=0))

In [ ]:
strategy_result

In [20]:
#port_df.to_csv('08-06-2021_port_df.csv',index=False)

In [22]:
df5 = pd.read_csv('08-06-2021_port_df.csv')

In [30]:
df5.tail()

,Date,ADANIPORTS,ASIANPAINT,AXISBANK,BAJAJ-AUTO,BAJAJFINSV,BAJFINANCE,BHARTIARTL,BPCL,BRITANNIA,CIPLA,COALINDIA,DIVISLAB,DRREDDY,EICHERMOT,GAIL,GRASIM,HCLTECH,HDFC,HDFCBANK,HDFCLIFE,HEROMOTOCO,HINDALCO,HINDUNILVR,ICICIBANK,INDUSINDBK,INFY,IOC,ITC,JSWSTEEL,KOTAKBANK,LT,M&M,MARUTI,NESTLEIND,NTPC,ONGC,POWERGRID,RELIANCE,SBILIFE,SBIN,SHREECEM,SUNPHARMA,TATAMOTORS,TATASTEEL,TCS,TECHM,TITAN,ULTRACEMCO,UPL,WIPRO,Total_Return
12,2021-05-20,10110.94,10412.40,10412.86,10232.96,10306.32,10286.01,10309.55,10183.24,10435.96,10435.79,10495.54,10174.32,10350.58,10259.84,10246.89,10340.20,10298.95,10365.84,10400.54,10445.37,10273.77,10484.68,10281.24,10401.04,10370.82,10182.21,10406.40,10244.09,10439.74,10395.22,10323.68,10547.73,10278.19,10306.32,10295.28,10461.84,10280.39,10350.47,10322.62,10268.88,10306.32,10356.16,10363.82,10487.04,10323.30,10274.92,10368.12,10242.85,10397.84,10331.57,516870.65
13,2021-05-21,10268.64,10345.72,10510.31,10338.99,10337.41,10453.54,10385.86,10142.59,10265.20,10302.98,10485.11,10321.69,10346.62,10353.37,10560.97,10230.52,10315.85,10431.97,10631.90,10348.51,10329.88,10408.13,10310.21,10505.09,10451.61,10331.95,10283.28,10381.02,10268.81,10469.61,10380.88,10468.45,10371.97,10337.41,10271.62,10375.53,10366.66,10379.66,10178.91,10446.09,10337.41,10314.45,10327.74,10330.39,10362.10,10298.41,10358.41,10339.02,10485.22,10223.39,518071.06
14,2021-05-24,10263.66,10328.03,10421.34,10407.32,10361.42,10344.59,10361.23,10361.56,10464.05,10389.91,10399.49,10427.58,10366.91,10362.06,10581.22,10510.03,10403.55,10228.66,10280.30,10296.02,10591.10,10308.90,10353.82,10418.52,10558.82,10329.15,10414.18,10499.60,10464.92,10250.47,10466.35,10309.34,10428.54,10361.42,10254.15,10410.92,10307.74,10398.12,10371.42,10248.01,10361.42,10307.61,10440.95,10227.68,10374.41,10434.92,10402.70,10352.70,10348.42,10410.02,518965.25
15,2021-05-25,10414.66,10437.17,10401.70,10347.54,10379.30,10374.63,10374.55,10199.25,10424.00,10373.14,10440.02,10347.44,10351.68,10315.22,10231.51,10550.03,10418.02,10467.38,10406.43,10556.45,10375.19,10373.32,10415.74,10291.96,10405.00,10414.44,10377.44,10401.99,10631.62,10378.00,10234.89,10446.26,10326.66,10379.30,10486.68,10605.20,10320.34,10335.65,10368.40,10360.55,10379.30,10238.04,10426.21,10505.93,10485.47,10556.60,10526.84,10385.13,10492.53,10497.84,520232.64
16,2021-05-26,10418.56,10436.87,10487.95,10446.21,10404.65,10516.63,10432.96,10221.11,10391.00,10435.34,10492.85,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [31]:
df5.Total_Return[15]

520232.64

In [33]:
allday_port_result.to_csv('08-06-2021_port_result.csv',index=False)

In [43]:
df6 = allday_port_result[allday_port_result['symbol']=='SBIN']
df6

,date,symbol,Total_win,Total_loss,No_of_trades,max_quantity,Total Return(%),capital_invested,capital_return,PL
39,2021-05-03,SBIN,0,0,0,0,0,10000,10000,0
89,2021-05-04,SBIN,2,2,4,27,0.2027,10000,9949.24,-50.76
139,2021-05-05,SBIN,1,5,6,28,-0.2945,10033.7,9911.91,-121.8
189,2021-05-06,SBIN,2,2,4,28,0.0342,10035.6,10065.5,29.96
239,2021-05-07,SBIN,2,3,5,28,-0.0576,10054.5,9980.83,-73.71
289,2021-05-10,SBIN,2,2,4,27,0.0783,10062.6,10088.5,25.92
339,2021-05-11,SBIN,3,1,4,28,-0.0112,10073.8,10212.7,138.91
389,2021-05-12,SBIN,4,1,5,27,0.1806,10118.2,10319.6,201.42
439,2021-05-14,SBIN,2,3,5,28,0.3408,10200.1,10258.9,58.8
489,2021-05-17,SBIN,2,1,3,27,-0.0853,10236,10548.4,312.39


In [50]:
df7 = df6.infer_objects()

In [52]:
df7.sum()

Total_win               30.0000
Total_loss              34.0000
No_of_trades            64.0000
max_quantity           402.0000
Total Return(%)          0.1699
capital_invested    162780.2900
capital_return      163198.3800
PL                     418.0900
dtype: float64